# Asthma-Like Illness Emergency Department Presentations (annual) - Processing

## Dependencies

Ensure that the required libraries have been installed locally as per the README.md file included in this project.

Run the following cell the import the required dependencies for this notebook.

In [10]:
# Import libraries
import pandas as pd

## Load Dataset

In [11]:
# Load the raw data.
df = pd.read_csv('raw.csv')

## Exploratory Analysis of Raw Data

In [12]:
# Generate summary statistics for object columns.
object_summary_stats = df.describe(include=['O']).transpose()                                                       # Generate summary statistics for object columns.
object_summary_stats['missing_values'] = df.isnull().sum()                                                          # Add missing values to the summary table.
object_summary_stats['present_values'] = df.notnull().sum()                                                         # Add present values to the summary table.
object_summary_stats['datatype'] = df.dtypes                                                                        # Add data types to the summary table.
object_summary_stats = object_summary_stats[['datatype', 'present_values', 'missing_values', 'unique']]             # Select features and reorder table.

# Display the summary tables with titles.
print("Dataset Head:")                                                                                              # Display the dataset head title.
display(df.head().style.set_table_styles([{'selector': 'th', 'props': [('min-width', '100px')]}]))                  # Display the dataset head. For better visualization, set the minimum width of the table headers to 100px.

print("\nObject Summary Statistics:")                                                                               # Display the object summary statistics.
display(object_summary_stats.style.set_table_styles([{'selector': 'th', 'props': [('min-width', '100px')]}]))       # Display the object summary statistics. For better visualization, set the minimum width of the table headers to 100px.

Dataset Head:


,Sex,LHD,Period,"Rate per 100,000 population",LL 95% CI,UL 95% CI
0,Males,Sydney LHD,14/15,283.100000,263.100000,304.1
1,Males,Sydney LHD,15/16,287.600000,267.500000,308.7
2,Males,Sydney LHD,16/17,263.700000,244.700000,283.8
3,Males,Sydney LHD,17/18,229.500000,211.800000,248.4
4,Males,Sydney LHD,18/19,244.400000,226.100000,263.7



Object Summary Statistics:


,datatype,present_values,missing_values,unique
Sex,object,432,420,3
LHD,object,432,420,16
Period,object,432,420,9
UL 95% CI,object,432,420,415


## Data Manipulation

In [13]:
# Rename columns to match Air Quality data set.
df = df.rename(columns={'LHD': 'lhd', 'Period': 'financial year', 'Sex': 'sex'})                                    # Rename columns
df.columns = df.columns.str.lower()                                                                                 # Lowercase column names

# Remove ' LHD' for Local Health District values.
df['lhd'] = df['lhd'].str.replace(' LHD', '')                                                                       # Remove ' LHD' from lhd column

# Remove rows representing state-wide aggregated data.
df = df.dropna(subset=['lhd'])                                                                                      # Remove rows with missing values in 'lhd' column
df = df[~df['lhd'].str.contains('All')]                                                                             # Remove rows with 'All' in 'lhd' column

# Remove columns holding Confidence Interval data.
df = df.loc[:, ~df.columns.str.contains('% ci')]                                                                    # Remove columns with '% ci' in the name

# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].apply(                                                                  # Add '20' to the start of each year value
    lambda x: f'20{x[:2]}/20{x[3:]}' if isinstance(x, str) else x
)

# View the cleaned data.
display(df.head())                                                                                                  # Display the first few rows of the cleaned data

,sex,lhd,financial year,"rate per 100,000 population"
0,Males,Sydney,2014/2015,283.1
1,Males,Sydney,2015/2016,287.6
2,Males,Sydney,2016/2017,263.7
3,Males,Sydney,2017/2018,229.5
4,Males,Sydney,2018/2019,244.4


### Reconfigure Table

In [14]:
# Pivot the dataframe to have 'sex' as columns
df = df.pivot_table(index=['financial year', 'lhd'], columns='sex', values='rate per 100,000 population').reset_index()

# Rename the columns to match the desired format
df.columns.name = None
df = df.rename(columns={
    'Persons': 'Persons rate per 100,000 population',
    'Males': 'Male rate per 100,000 population', 
    'Females': 'Female rate per 100,000 population'
})

# View the DataFrame
display(df.head()) 

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2014/2015,Central Coast,373.6,354.3,366.6
1,2014/2015,Far West,771.9,603.0,690.5
2,2014/2015,Hunter New England,455.6,445.5,452.6
3,2014/2015,Illawarra Shoalhaven,410.1,383.6,398.2
4,2014/2015,Mid North Coast,495.6,478.8,490.2


## Set Date Range

Set the range of financial years from 2014/2015 to 2023/2024.

In [15]:
# Add rows for each LHD for the missing years until 2023/2024.
missing_rows = []                                                                                                   # Create a list to store the missing rows.
lhds = df['lhd'].unique()                                                                                           # Get the unique values in the 'lhd' column.
years = [f"{year}/{year + 1}" for year in range(2014, 2024)]                                                        # Create a list of years from 2014/2015 to 2023/2024.

for lhd in lhds:
    for year in years:
        if not ((df['lhd'] == lhd) & (df['financial year'] == year)).any():                                         # Check if the row is missing.
            missing_rows.append({                                                                                   # Append the missing row to the list.
                'lhd': lhd, 
                'financial year': year, 
                'Female rate per 100,000 population': None,
                'Male rate per 100,000 population': None,
                'Persons rate per 100,000 population': None,

            })          

# Create a DataFrame from the missing rows and concatenate it to the original DataFrame
if missing_rows:
    df_missing = pd.DataFrame(missing_rows)                                                                         # Create a DataFrame from the missing rows.
    df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.

# Sort the DataFrame by 'lhd' and 'financial year'.
df = df.sort_values(by=['lhd', 'financial year']).reset_index(drop=True)                                            # Sort the DataFrame by 'lhd' and 'financial year'.

# View the DataFrame.
display(df.tail())                                                                                                  # View the last 5 rows of the DataFrame.

C:\Users\matth\AppData\Local\Temp\ipykernel_18844\2552393688.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.


,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
145,2019/2020,Western Sydney,244.8,203.2,224.7
146,2020/2021,Western Sydney,184.8,188.5,187.3
147,2021/2022,Western Sydney,187.4,184.0,186.1
148,2022/2023,Western Sydney,220.4,209.3,215.7
149,2023/2024,Western Sydney,NaN,NaN,NaN


Fill missing values using linear interpolation.

In [16]:
df['Female rate per 100,000 population'] = df['Female rate per 100,000 population'].interpolate()                       # Fill missing values using linear interpolation.
df['Male rate per 100,000 population'] = df['Male rate per 100,000 population'].interpolate()                           # Fill missing values using linear interpolation.
df['Persons rate per 100,000 population'] = df['Persons rate per 100,000 population'].interpolate()                     # Fill missing values using linear interpolation.

## Output Processed Dataset

In [17]:
# Save the processed data to a new CSV file.
df.to_csv('processed.csv', index=False)                        # Save the processed data to a new CSV file.                                                                                                      # View the first 5 rows of the DataFrame.

## View Processed Dataset

In [18]:
# Display the processed data.
display(df.head())

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2014/2015,Central Coast,373.6,354.3,366.6
1,2015/2016,Central Coast,405.8,350.7,380.9
2,2016/2017,Central Coast,412.0,339.7,378.4
3,2017/2018,Central Coast,386.7,344.0,368.2
4,2018/2019,Central Coast,403.8,349.0,378.8
